## Comparing Agents List:

###### 1. Single Action Agent
Always returns the same action / product id.

###### 2. Random Action Agent
The Agent returns the random action from all of product id's range.

###### 3. Organic Product Count Agent
The Agent that selects an action based on the most frequently viewed Product.

###### 4. Organic Per User Count Agent
The Agent that counts organic views of products per a user and selects an action for the most frequently shown product.

###### 5. Define Organic MF Agent
The Agent that selects an action from the model that performs Organic Events matrix factorisation.

###### 6. Bandit Click Count Agent
The Agent that selects an Action for the most frequently clicked bandit action before.

###### 7. Bandit MF Agent - used Matrix Factorization learning model
The Agent chooses maximum logit (logistic regression) from all possible actions.

###### 8. Likelihood Agent
Act method returns an action based on current observation and past history.

## Environment Setup

In [ ]:
from copy import deepcopy

import recogym
from my_agent_utils import add_agent_id, combine_stat
from my_entries.likehood_agent import LikelihoodAgent, CountFeatureProvider
from single_action_agent import SingleActionAgent
%matplotlib inline
import gym
import matplotlib.pyplot as plt

from recogym import env_1_args, Configuration
from recogym.agents import OrganicUserEventCounterAgent, organic_user_count_args, OrganicCount, organic_count_args, \
    OrganicMFSquare, organic_mf_square_args, BanditCount, bandit_count_args, BanditMFSquare, bandit_mf_square_args
from recogym.agents import RandomAgent, random_args
from recogym.evaluate_agent import plot_verify_agents, verify_agents

# Set style for pretty plots
plt.style.use('ggplot')

products = 40

offline_users = 16000
online_users = 200

# You can overwrite environment arguments here:

env_1_args['random_seed'] = 42
env_1_args['num_products']= products
env_1_args['phi_var']=0.0
env_1_args['number_of_flips']=1

# env_1_args['sigma_mu_organic'] = 0.0
# env_1_args['sigma_omega']=0
# env_1_args['normalize_beta']=True

# Initialize the gym for the first time by calling .make() and .init_gym()

#---------------------------------------------
%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = [6, 3]

env_1_args['number_of_flips'] = 0
env_1_args['sigma_mu_organic'] = 0.0
env_1_args['sigma_omega'] = 1
env_1_args['random_seed'] = 42
env_1_args['num_products'] = products
env_1_args['K'] = 5
env_1_args['number_of_flips'] = 5
#---------------------------------------------

env = gym.make('reco-gym-v1')
env.init_gym(env_1_args)

env.reset()


1. Define Single Action Agent

In [ ]:
single_action_agent = SingleActionAgent(Configuration({**env_1_args,}))

2. Define Random Action Agent

In [ ]:
random_agent = RandomAgent(Configuration({**random_args,**env_1_args,}))

3. Define Organic Product Count Agent

In [ ]:
organic_count_agent = OrganicCount(Configuration({**organic_count_args,**env_1_args,}))

4. Define Organic Per User Count Agent

In [ ]:
organic_user_counter_agent = OrganicUserEventCounterAgent(Configuration({**organic_user_count_args, **env_1_args, 'select_randomly': True,}))

5. Define Organic MF Agent

In [ ]:
organic_mf_agent = OrganicMFSquare(Configuration({**organic_mf_square_args, **env_1_args, 'select_randomly': True,}))

6. Define Bandit Click Count Agent

In [ ]:
bandit_count_agent = BanditCount(Configuration({**bandit_count_args,**env_1_args,}))

7. Define Bandit MF Agent

In [ ]:
bandit_mf_square_agent = BanditMFSquare(Configuration({ **bandit_mf_square_args, **env_1_args,}))

8. Define Likelihood Agent

In [ ]:
from scipy.stats import beta
import pandas as pd

config = Configuration(env_1_args)

count_product_views_feature_provider = CountFeatureProvider(config)

likelihood_agent = LikelihoodAgent(count_product_views_feature_provider)

lh_data = deepcopy(env).generate_logs(offline_users)

likelihood_agent.train(lh_data)

bandits = lh_data[lh_data['z'] == 'bandit']
successes = bandits[bandits['c'] == 1].shape[0]
failures = bandits[bandits['c'] == 0].shape[0]

lh_agent_res = add_agent_id(
    'Likelihood',
    beta.ppf(0.025, successes + 1, failures + 1),
    beta.ppf(0.500, successes + 1, failures + 1),
    beta.ppf(0.975, successes + 1, failures + 1))


In [ ]:
pd.DataFrame().from_dict(lh_agent_res)

## A/B-Test Evaluation

In [ ]:
result_bandit_mf = recogym.test_agent(deepcopy(env), deepcopy(bandit_mf_square_agent), offline_users, online_users)
result_bandit_mf_id = add_agent_id('Bandit MF Agent', *result_bandit_mf)

In [ ]:
result_single = recogym.test_agent(deepcopy(env), deepcopy(single_action_agent), offline_users, online_users)
result_single_id = add_agent_id('Single Action Agent', *result_single)

In [ ]:
result_random = recogym.test_agent(deepcopy(env), deepcopy(random_agent), offline_users, online_users)
result_random_id = add_agent_id('Random Agent', *result_random)

In [ ]:
result_organic_count = recogym.test_agent(deepcopy(env), deepcopy(organic_count_agent), offline_users, online_users)
result_organic_count_id = add_agent_id('Organic Count Agent', *result_organic_count)

In [ ]:
result_organic_user_counter = recogym.test_agent(deepcopy(env), deepcopy(organic_user_counter_agent), offline_users, online_users)
result_organic_user_counter_id = add_agent_id('Organic Count per User Agent', *result_organic_user_counter)

In [ ]:
result_organic_mf = recogym.test_agent(deepcopy(env), deepcopy(organic_mf_agent), offline_users, online_users)
result_organic_mf_id = add_agent_id('Organic MF Agent', *result_organic_mf)

In [ ]:
result_bandit_count = recogym.test_agent(deepcopy(env), deepcopy(bandit_count_agent), offline_users, online_users)
result_bandit_count_id = add_agent_id('Bandit Count Agent', *result_bandit_count)

In [ ]:
result_likelihood_id = verify_agents(
    env,
    number_of_users=online_users,
    agents={
        'Likelihood LogReg': deepcopy(likelihood_agent),
    }
)


In [ ]:
comb_result = combine_stat([
                            result_single_id,
                            result_random_id,
                            result_organic_count_id,
                            result_organic_user_counter_id,
                            result_organic_mf_id,
                            result_bandit_mf_id,
                            result_likelihood_id
                            ])


In [ ]:
comb_result

In [ ]:
fig = plot_verify_agents(comb_result)
plt.ylabel('CTR')
plt.show()




